In [8]:
import os
import time
import sys
import textwrap
import subprocess

# --- 0. Install Dependencies (Crucial Fix for 'streamlit' not found error) ---
print("Checking and installing necessary dependencies...")
# List of packages needed for both the deployment (pyngrok) and the Streamlit app
required_packages = ["streamlit", "pyngrok", "pandas", "numpy", "matplotlib", "seaborn"]

try:
    # Use pip to install or upgrade the packages
    for package in required_packages:
        print(f"Attempting to install/upgrade {package}...")
        # Use sys.executable to ensure we use the correct Python interpreter's pip
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", package],
                                stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print("All dependencies installed/verified successfully.")
except Exception as e:
    print(f"FATAL: Failed to install one or more dependencies. The app cannot run. Error: {e}")
    # Exit the script if dependencies fail to install
    sys.exit(1)

# Import pyngrok and conf after it's confirmed to be installed
from pyngrok import ngrok, conf # <<-- FIX 1: Import 'conf' to override default version

# --- 1. تعريف محتوى ملف Streamlit بالكامل (app.py) ---
# هذا هو محتوى ملف app.py الذي سيتم كتابته
raw_app_content = """
# ----------------------------------------------------------------------
# Production Line Downtime Risk Simulator & Dashboard (English Version)
# ----------------------------------------------------------------------

import streamlit as st
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# نستخدم نمط فاتح للرسوم البيانية (مثل ggplot) ليتناسب مع المظهر الفاتح
plt.style.use('ggplot')

# --- 1. Load Model and Features ---
# A mock model is included for display purposes if the pkl files are not present.
try:
    with open('random_forest_model.pkl', 'rb') as file:
        model = pickle.load(file)
    with open('model_features.pkl', 'rb') as file:
        model_features = pickle.load(file)
except FileNotFoundError:
    class MockModel:
        def predict(self, df):
            # Simulation of prediction based on inputs
            incident = df['Incident_Count'].iloc[0] if 'Incident_Count' in df.columns else 0
            sequence = df['Operator_Batch_Sequence_Count'].iloc[0] if 'Operator_Batch_Sequence_Count' in df.columns else 0
            prev_downtime = df['Avg_Downtime_Prev_3_Batches'].iloc[0] if 'Avg_Downtime_Prev_3_Batches' in df.columns else 0
            # Increased base deviation for visual impact
            return np.array([7 + incident * 3.5 + sequence * 2.0 + prev_downtime * 0.6])

    model = MockModel()
    model_features = ['Is_High_Risk_Failure', 'Incident_Count', 'Operator_Batch_Sequence_Count', 'Avg_Downtime_Prev_3_Batches']

# --- 2. Page Configuration and Sidebar ---
st.set_page_config(
    page_title="Production Risk Dashboard",
    page_icon="🏭",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- Injecting Custom Professional Light-Themed CSS ---
st.markdown(
    '''
<style>
/* 1. Global Background and Text (Professional Light Theme) */
body {
    color: #333333; /* Dark text */
    background-color: #F0F2F6; /* Light gray background */
}
.stApp {
    background-color: #F0F2F6;
}

/* 2. Main Title Styling (Corporate Blue accent and border) */
h2 {
    color: #1E90FF; /* Corporate Blue */
    font-weight: 700;
    border-left: 5px solid #1E90FF;
    padding-left: 10px;
    margin-top: 10px;
}

/* 3. Sidebar Styling */
[data-testid="stSidebar"] {
    background-color: #FFFFFF; /* Pure white sidebar */
    box-shadow: 2px 0 5px rgba(0, 0, 0, 0.1);
}

/* 4. Container/Card Styling (Adding lift/depth) */
.stContainer {
    background-color: #FFFFFF; /* White card background */
    border-radius: 12px;
    border: 1px solid #DEDEDE; /* Light border */
    box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08); /* Soft shadow */
    padding: 20px;
    margin-bottom: 20px;
}

/* 5. Metric (KPI) Values */
[data-testid="stMetricValue"] {
    color: #1E90FF; /* KPI values in accent color (Blue) */
    font-size: 2.2rem;
    font-weight: 700;
}

/* 6. Primary Button Styling (Prominent Blue) */
.stButton>button {
    background-color: #1E90FF;
    color: white;
    font-weight: bold;
    border-radius: 8px;
    border: none;
    padding: 10px 20px;
    transition: background-color 0.3s;
}
.stButton>button:hover {
    background-color: #1565C0; /* Darker blue on hover */
}

/* 7. Input/Select box backgrounds (Clean, subtle look) */
div[data-testid="stSelectbox"] > div[role="combobox"],
div[data-testid="stNumberInput"] > div > div,
div[data-testid="stSlider"] {
    background-color: #FFFFFF; /* White input fields */
    border-radius: 6px;
    padding: 5px;
    border: 1px solid #CCCCCC;
}

/* Ensure all text is dark */
.stMarkdown, .stText, p {
    color: #333333;
}

</style>
    ''',
    unsafe_allow_html=True
)

# Sidebar for branding and info
st.sidebar.markdown("# ⚙️ Predictive Operations")
st.sidebar.markdown("### Risk Mitigation Dashboard")
st.sidebar.info("Analyze risk factors and simulate batch performance deviations.")
st.sidebar.markdown("---")
st.sidebar.caption("Powered by Streamlit & Mock ML Model")


st.markdown("## ✨ Production Line Cycle Time Risk Assessment", unsafe_allow_html=True)
st.markdown("---")
st.markdown('''
    This **Predictive Dashboard** provides a **diagnostic risk assessment** based on key operational factors to simulate and mitigate potential cycle time deviation.
''')

# --- 3. Mock Model Warning ---
if type(model).__name__ == 'MockModel':
    st.error("⚠️ Warning: Mock Model is Active. Load 'random_forest_model.pkl' for production use.")

# --- 4. Main Navigation (Tabs) ---
tab1, tab2, tab3 = st.tabs(["Dashboard & Summary 📈", "Scenario Analysis 🔬", "Final Recommendations 📋"])

# Define dark color for plot text/ticks for the light theme
PLOT_TEXT_COLOR = '#333333'
# Define light color for plot background (Streamlit default)
PLOT_BG_COLOR = 'white'


# ************** Tab 1: Dashboard & Summary **************

with tab1:
    st.header("📊 Key Diagnostic Results Summary")

    # KPIs in a visually distinct container
    with st.container(border=True):
        st.subheader("Operational Key Performance Indicators (KPIs)")
        kpi_col1, kpi_col2, kpi_col3, kpi_col4 = st.columns(4)

        kpi_col1.metric("Highest Cost Driver", "Micro-Stops (45%)", delta="-5% MoM", delta_color="inverse")
        kpi_col2.metric("Least Consistent Operator", "Operator D", delta="+2.1% Inconsistency", delta_color="inverse")
        kpi_col3.metric("Model R² (Diagnostic)", "0.72", delta="Upgraded", delta_color="normal")
        kpi_col4.metric("Avg Batch Deviation", "12.5 min", delta="0.3 min", delta_color="inverse")

    st.markdown("---")

    # Charts Section
    st.subheader("Detailed Visual Analysis")

    chart_col1, chart_col2 = st.columns(2)

    with chart_col1:
        # Mock Feature Importance Chart (Chart 1) - Using 'Blues' for light background contrast
        st.caption("Feature Importance: Impact on Cycle Time Deviation")
        fig1, ax1 = plt.subplots(figsize=(6, 4))
        features = ['Micro-Stops', 'Operator Sequence', 'Prev Downtime', 'High Risk Flag']
        importance = [0.45, 0.25, 0.20, 0.10]
        sns.barplot(x=importance, y=features, ax=ax1, palette="Blues_r")
        ax1.set_title('Predictive Feature Importance', fontsize=12, color=PLOT_TEXT_COLOR)
        ax1.set_xlabel('Relative Importance Score', color=PLOT_TEXT_COLOR)
        ax1.set_ylabel('')
        # Set text/tick colors to dark for light theme
        ax1.tick_params(axis='x', colors=PLOT_TEXT_COLOR)
        ax1.tick_params(axis='y', colors=PLOT_TEXT_COLOR)
        fig1.patch.set_facecolor(PLOT_BG_COLOR)
        ax1.patch.set_facecolor(PLOT_BG_COLOR)
        st.pyplot(fig1)

    with chart_col2:
        # Mock Downtime Pareto Chart (Chart 2) - Using 'Set2' for light background contrast
        st.caption("Downtime Causes by Frequency (Cost Drivers)")
        fig2, ax2 = plt.subplots(figsize=(6, 4))
        causes = ['Tooling Change', 'Sensor Error', 'Material Jam', 'Operator Break', 'Others']
        costs = [50, 25, 15, 5, 5]
        sns.barplot(x=causes, y=costs, ax=ax2, palette="Set2")
        ax2.set_title('Pareto Analysis Mock', fontsize=12, color=PLOT_TEXT_COLOR)
        ax2.set_ylabel('Frequency (%)', color=PLOT_TEXT_COLOR)
        ax2.set_xlabel('Cause', color=PLOT_TEXT_COLOR)
        # Set text/tick colors to dark for light theme
        ax2.tick_params(axis='x', rotation=15, colors=PLOT_TEXT_COLOR)
        ax2.tick_params(axis='y', colors=PLOT_TEXT_COLOR)
        fig2.patch.set_facecolor(PLOT_BG_COLOR)
        ax2.patch.set_facecolor(PLOT_BG_COLOR)
        st.pyplot(fig2)

    # Mock Operator Consistency Chart (Chart 3)
    st.markdown("### Operator Performance Consistency")
    st.caption("Coefficient of Variation (C.V.) in Batch Time - Lower is Better (Target < 2.0%)")
    fig3, ax3 = plt.subplots(figsize=(12, 4))
    operators = ['A', 'B', 'C', 'D', 'E']
    consistency = np.random.uniform(1.0, 5.0, len(operators))
    sns.barplot(x=operators, y=consistency, ax=ax3, palette="cividis")
    ax3.set_title('Operator Batch Cycle Time Consistency', fontsize=12, color=PLOT_TEXT_COLOR)
    ax3.set_ylabel('Coefficient of Variation (%)', color=PLOT_TEXT_COLOR)
    ax3.set_xlabel('Operator ID', color=PLOT_TEXT_COLOR)
    ax3.axhline(2.0, color='red', linestyle='--', linewidth=1, label='Target C.V. (2.0%)')
    ax3.legend(loc='upper right')
    # Set text/tick colors to dark for light theme
    ax3.tick_params(axis='x', colors=PLOT_TEXT_COLOR)
    ax3.tick_params(axis='y', colors=PLOT_TEXT_COLOR)
    fig3.patch.set_facecolor(PLOT_BG_COLOR)
    ax3.patch.set_facecolor(PLOT_BG_COLOR)
    st.pyplot(fig3)

    st.info("💡 Insight: Operator D's inconsistency suggests a need for targeted training or process standardization.")


# ************** Tab 2: Scenario Analysis **************

with tab2:
    st.header("🔬 Single Batch Risk Simulation")
    st.markdown("Evaluate the impact of changing operational parameters on expected cycle time deviation.")

    # Input Section
    with st.container(border=True):
        st.subheader("Current Batch Operational Parameters:")

        input_col1, input_col2 = st.columns(2)

        with input_col1:
            is_high_risk = st.selectbox(
                "1. Is this a High-Risk Batch (e.g., New product / Unstable line)?",
                [0, 1],
                format_func=lambda x: "Yes" if x == 1 else "No",
                help="Set to 'Yes' if running a new SKU or after recent maintenance."
            )
            incident_count = st.number_input(
                "2. Expected Count of Micro-Stops/Incidents:",
                min_value=0,
                value=2,
                max_value=10,
                help="Estimate the number of small incidents (micro-stops, jams) in the upcoming batch."
            )

        with input_col2:
            # Using a Slider for better UX on fatigue measure
            operator_sequence = st.slider(
                "3. Operator Batch Sequence Count (Fatigue Measure):",
                min_value=1,
                max_value=10,
                value=5,
                step=1,
                help="The sequence number of the batch for the assigned operator during the current shift (higher = more fatigue/risk)."
            )
            avg_downtime_prev = st.number_input(
                "4. Average Downtime from Previous 3 Batches (Minutes):",
                min_value=0.0,
                value=15.0,
                step=0.5,
                help="Reflects recent line stability. High recent downtime suggests residual issues."
            )

    st.markdown("---")

    if st.button("Predict Cycle Time Deviation", type="primary"):
        # 1. Prepare input DataFrame
        input_data = pd.DataFrame(0, index=[0], columns=model_features)

        # 2. Match user inputs
        if 'Is_High_Risk_Failure' in input_data.columns:
            input_data['Is_High_Risk_Failure'] = is_high_risk
        if 'Incident_Count' in input_data.columns:
            input_data['Incident_Count'] = incident_count
        if 'Operator_Batch_Sequence_Count' in input_data.columns:
            input_data['Operator_Batch_Sequence_Count'] = operator_sequence
        if 'Avg_Downtime_Prev_3_Batches' in input_data.columns:
            input_data['Avg_Downtime_Prev_3_Batches'] = avg_downtime_prev

        # 3. Run Prediction
        try:
            prediction = model.predict(input_data)[0]

            # 4. Display Results in a results container
            with st.container(border=True):
                st.subheader("Prediction Result")

                result_col1, result_col2 = st.columns([1, 2])

                result_col1.metric(
                    label="Predicted Cycle Time Deviation",
                    value=f"{prediction:.1f} minutes",
                    delta_color="off"
                )

                # Risk Assessment based on prediction
                if prediction > 25:
                    result_col2.error("🚨 CRITICAL RISK (Time Overrun > 25 min)! Immediate intervention required. Stop and Inspect.")
                    st.markdown("**Required Action:** Divert batch to a less critical line, assign a relief operator, and escalate maintenance review.")
                elif prediction > 15:
                    result_col2.warning("🔶 HIGH RISK (Time Overrun 15-25 min). Proceed with caution and increase operator supervision.")
                    st.markdown("**Required Action:** Consider a quick line check (5-10 min) and assign a rested operator.")
                elif prediction > 10:
                    result_col2.info("🟡 MODERATE RISK (Time Overrun 10-15 min). Monitor closely.")
                else:
                    result_col2.success("🟢 LOW RISK (Time Overrun < 10 min). Performance expected within acceptable limits.")

        except Exception as e:
            st.error(f"Prediction Error: Details: {e}")


# ************** Tab 3: Final Recommendations **************

with tab3:
    st.header("📋 Actionable Recommendations")
    st.markdown("---")

    st.markdown('''## Strategic Mitigation Plan''')
    st.info("These are high-level, data-driven recommendations derived from the overall diagnostic analysis.")

    st.subheader("1. Operations Management & Priority Focus")
    st.markdown('''
    - **Micro-Stop Reduction:** The primary focus must be on eliminating micro-stops. This accounts for ~45% of the predictive risk score. Implement a **zero-tolerance** policy on unrecorded short stops.
    - **KPI Target:** Reduce the average batch deviation to below 10 minutes within the next quarter.
    ''')

    st.subheader("2. Human Factors & Training")
    st.markdown('''
    - **Fatigue Mitigation:** Implement a system to **limit consecutive batch assignments** to no more than 4 batches per operator per shift. High sequence counts significantly increase human-error risk.
    - **Targeted Training:** Provide specialized training for **Operator D** (and other low-consistency operators) focusing on standardized machine setup and changeover procedures.
    ''')

    st.subheader("3. Preventive Maintenance (PM) Strategy")
    st.markdown('''
    - **Proactive Threshold:** Establish a rule for mandatory quick line inspection if the `Average Downtime from Previous 3 Batches` exceeds **18 minutes**. This prevents cascading failures.
    - **Sensor Review:** Investigate the root cause of 'Sensor Error' downtime (25% of incidents) to transition from reactive fixes to proactive sensor calibration.
    ''')
"""

streamlit_app_content = textwrap.dedent(raw_app_content)

# --- 2. كتابة ملف app.py باستخدام Python القياسي (Writing app.py using standard Python) ---
print("Writing app.py...")
with open('app.py', 'w', encoding='utf-8') as f:
    f.write(streamlit_app_content)
print("app.py written successfully.")


# تم وضع المفتاح الجديد الذي زودتِني به (The new key you provided is placed here)
NGROK_AUTH_TOKEN = "35yOpo3SS2yc4FmLEz0Mx0gsOx2_3pCfP24A88ZPwTLv5s7b3"

# إيقاف أي أنفاق ngrok سابقة (Kill any previous ngrok tunnels)
try:
    ngrok.kill()
except:
    pass

# --- FIX 2: Explicitly set the default version to '4' to prevent the 404 error from v3 download URL ---
conf.DEFAULT_NGROK_VERSION = '4'

# Ensure ngrok binary is not present to force a fresh download
ngrok_path = ngrok.conf.get_default().ngrok_path
if os.path.exists(ngrok_path):
    print(f"Removing existing ngrok binary at {ngrok_path} to force fresh download.")
    os.remove(ngrok_path)

# --- Removed the problematic explicit ngrok.install_ngrok call. set_auth_token handles it now. ---
print("Configuring ngrok version 4 for installation...")


# Now set the auth token after ensuring ngrok is configured to install version 4
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# تشغيل Streamlit في الخلفية (Running Streamlit in background)
print("Starting Streamlit app in background using subprocess.Popen...")

try:
    # Use Popen to start the Streamlit process. This is more robust than os.system with '&'.
    streamlit_process = subprocess.Popen(
        ["streamlit", "run", "app.py"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )

    # الانتظار 8 ثواني لضمان بدء Streamlit بالكامل (Wait 8 seconds for Streamlit to fully start)
    print("Waiting 8 seconds for Streamlit to launch on port 8501...")
    time.sleep(8)

    # الاتصال عبر ngrok (Connecting عبر ngrok)
    print("Creating public URL...")

    # ngrok.connect returns a list of tunnels, we take the first one
    public_url = ngrok.connect(8501).public_url

    print("\n")
    print("----------------------------------------------------------")
    print(f"✅ Streamlit App is running successfully!")
    print(f"Public URL (Shareable on Mobile!): {public_url}")
    print("----------------------------------------------------------")
    print("\n")

except Exception as e:
    print(f"❌ ngrok connection failed. Details: {e}")
    # Attempt to terminate the Streamlit process on failure
    if 'streamlit_process' in locals() and streamlit_process:
        print("Attempting to terminate Streamlit process...")
        streamlit_process.terminate()

finally:
    if 'streamlit_process' in locals() and streamlit_process:
        print(f"Streamlit PID: {streamlit_process.pid}. Keep this script running to maintain the public URL.")

Checking and installing necessary dependencies...
Attempting to install/upgrade streamlit...
Attempting to install/upgrade pyngrok...
Attempting to install/upgrade pandas...
Attempting to install/upgrade numpy...
Attempting to install/upgrade matplotlib...
Attempting to install/upgrade seaborn...
All dependencies installed/verified successfully.
Writing app.py...
app.py written successfully.
Configuring ngrok version 4 for installation...
Starting Streamlit app in background using subprocess.Popen...
Waiting 8 seconds for Streamlit to launch on port 8501...
Creating public URL...


----------------------------------------------------------
✅ Streamlit App is running successfully!
Public URL (Shareable on Mobile!): https://victoria-prepurposive-omar.ngrok-free.dev
----------------------------------------------------------


Streamlit PID: 5046. Keep this script running to maintain the public URL.
